In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

Cloning into 'KoBERT'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 179.81 KiB | 821.00 KiB/s, done.
Resolving deltas: 100% (82/82), done.


In [ ]:
cd KoBERT

/content/KoBERT


In [ ]:
!pip install -r requirements.txt
!pip install .

     |████████████████████████████████| 55.0MB 73kB/s 
     |████████████████████████████████| 348kB 63.9MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 
     |████████████████████████████████| 4.1MB 44.2MB/s 
     |████████████████████████████████| 1.5MB 41.0MB/s 
     |████████████████████████████████| 2.9MB 38.5MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588507 sha256=d37d188caae26278933a63a4daac734967351f582153424626645876d61b6273
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=041ab04a74ef38bf19cf9b895a72d15f3e89dbbb650d5af4abe010f08941eecf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built gluonnlp sacremoses
  Found existing installatio

In [ ]:
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [ ]:
# 모듈 임포트

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
import os
import pickle
import gensim
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dropout, Dense
from tensorflow.keras import regularizers

In [ ]:
tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)

[██████████████████████████████████████████████████]


In [ ]:
os.chdir('/content/리뽀/')

In [ ]:
train_data = pd.read_csv('1. Data/news_train.csv')

x_train_data = train_data['content']
y_train_data = train_data['info']

# 콘텐츠 가져와서 형태소분석뒤 다시저장
for i in range(0, len(x_train_data)):
  x_train_data[i] = sp(x_train_data[i])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train_data)
x_Train_sequences = tokenizer.texts_to_sequences(x_train_data)

vocabulary = tokenizer.word_index
vocab_size = len(vocabulary) + 1

Train_data = x_Train_sequences

max_len = 25
X_train = pad_sequences(Train_data, maxlen = max_len)
y_train = y_train_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 토크나이저 저장

with open('3. Tokenizer/tokenizer.pickle', 'wb') as handle:  
  pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('4. Pre_trained embedding/GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))     # 300차원의 임베딩 매트릭스 생성
cnt = 0

for index, word in enumerate(vocabulary):          # vocabulary에 있는 토큰들을 하나씩 넘김
  if word in word2vec:                             # 넘겨받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 의미)
    embedding_vector = word2vec[word]              # 해당 토큰에 해당하는 vector를 불러오고
    embedding_matrix[index] = embedding_vector     # 해당 위치의 embedding_matrix에 저장
  else:
    cnt += 1
    continue

In [ ]:
np.save('4. Pre_trained embedding/embedding_matrix.npy', embedding_matrix)

In [ ]:
max_len = 25

model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_len))  # 임베딩 가중치 적용 코드
model.add(SpatialDropout1D(0.4))
model.add(LSTM(32))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')
print(model.summary())

history = model.fit(X_train, y_train, epochs = 1, batch_size = 32)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 300)           2406900   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,450,613
Trainable params: 2,450,613
Non-trainable params: 0
____________________________________________

In [ ]:
model.save('5. Model/kobert_gensim_model.h5')   

Test 시간재기 시작


In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

In [ ]:
cd KoBERT

In [ ]:
!pip install -r requirements.txt
!pip install .

In [ ]:
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [ ]:
# 데이터 불러오기

import pandas as pd

test_data = pd.read_csv('1. Data/news_test.csv')
submission_data = pd.read_csv('1. Data/sample_submission.csv')

In [ ]:
# 시간 측정 시작

import time
start = time.time()  # 시작 시간 저장

In [ ]:
# Library 불러오기

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
import pickle
from tensorflow.keras.models import load_model

In [ ]:
# pos_tagger, tokenizer, pretrained_embedding, model 불러오기

tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)                              # pos_tagger

with open('3. Tokenizer/tokenizer.pickle', 'rb') as handle:        # tokenizer
       loaded_tokenizer = pickle.load(handle)

np.load('4. Pre_trained embedding/embedding_matrix.npy')           # pre_trained embedding

loaded_model = load_model('5. Model/kobert_gensim_model.h5')       # model

using cached model


In [ ]:
# 형태소 분석 + 전처리

x_test_data = test_data['content']

for i in range(0, len(x_test_data)):
  x_test_data[i] = sp(x_test_data[i])
  
tokenizer = loaded_tokenizer
x_Test_sequences = tokenizer.texts_to_sequences(x_test_data)

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1

vocabulary = tokenizer.word_index
Test_data = x_Test_sequences

max_len = 25
X_test = pad_sequences(Test_data, maxlen = max_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 예측

pred_test = loaded_model.predict(X_test)
submission_data.loc[:,'info'] = np.where(pred_test > 0.5, 1,0).reshape(-1)

In [ ]:
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 263.28223943710327


In [ ]:
# 제출

submission_data.loc[:, ["id","info"]].to_csv("submission_data(KoBERT)(gensim)1228(noval)ep1-newpara.csv", index = False)
submission_data

,id,info
0,NEWS00237_1,1
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
